In [4]:
import pandas as pd
from datetime import date, datetime, timedelta

# AIMS

1. Ratio de endeudamiento
2. Suma de cuota mensual monetaria por cada credito

## 1. Ratio de endeudamiento por cliente general

In [15]:
creditos = pd.read_csv('creditos.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
creditos.head()

,id_cliente,id_prestamo,monto,cuotas,deuda_actual,fecha_inicio,fecha_pago,fecha_pago.1,dias_atraso
0,1,10001,100,10,80,12/12/2022,12/12/2022,12/12/2022,0
1,1,10002,200,5,150,22/12/2022,22/12/2022,23/12/2022,1
2,2,10003,50,10,5,23/12/2022,23/12/2022,28/12/2022,5
3,2,10004,75,3,70,24/12/2022,24/12/2022,28/12/2022,4
4,2,10005,125,5,12,24/12/2022,24/12/2022,29/12/2022,5


In [16]:
clientes_agrupados = creditos.groupby('id_cliente').sum()
clientes_agrupados

C:\Users\Oscar\AppData\Local\Temp\ipykernel_7456\2360885637.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  clientes_agrupados = creditos.groupby('id_cliente').sum()


,id_prestamo,monto,cuotas,deuda_actual,dias_atraso
id_cliente,,,,,
1,40017,650,22,241,4
2,40019,650,28,109,18


In [17]:
clientes_agrupados['ratio_endeudamiento'] = clientes_agrupados['deuda_actual'] / clientes_agrupados['monto']
clientes_agrupados

,id_prestamo,monto,cuotas,deuda_actual,dias_atraso,ratio_endeudamiento
id_cliente,,,,,,
1,40017,650,22,241,4,0.370769
2,40019,650,28,109,18,0.167692


## 2. Suma de cuota mensual monetaria por cada credito

In [7]:
# funcion mes
def get_month(x):
    return datetime(x.year, x.month, 1)

In [8]:
creditos = pd.DataFrame({
    "Cliente": ["Juan", "Juan", "Ana", "Ana", "Pedro", "Pedro"],
    "Monto": [1000, 5000, 2000, 3000, 1500, 2000],
    "Cuotas": [12, 24, 6, 12, 8, 12],
    "Fecha": ["2022-01-01", "2022-02-01", "2022-01-01", "2022-03-01", "2022-01-01", "2022-02-01"]
})
creditos["Fecha"] = pd.to_datetime(creditos["Fecha"])
creditos['fecha_mes'] = creditos['Fecha'].apply(get_month)
creditos.set_index("fecha_mes", inplace=True)
creditos

,Cliente,Monto,Cuotas,Fecha
fecha_mes,,,,
2022-01-01,Juan,1000,12,2022-01-01
2022-02-01,Juan,5000,24,2022-02-01
2022-01-01,Ana,2000,6,2022-01-01
2022-03-01,Ana,3000,12,2022-03-01
2022-01-01,Pedro,1500,8,2022-01-01
2022-02-01,Pedro,2000,12,2022-02-01


In [9]:
creditos["Cuota Mensual"] = creditos["Monto"] / creditos["Cuotas"]
creditos

,Cliente,Monto,Cuotas,Fecha,Cuota Mensual
fecha_mes,,,,,
2022-01-01,Juan,1000,12,2022-01-01,83.333333
2022-02-01,Juan,5000,24,2022-02-01,208.333333
2022-01-01,Ana,2000,6,2022-01-01,333.333333
2022-03-01,Ana,3000,12,2022-03-01,250.000000
2022-01-01,Pedro,1500,8,2022-01-01,187.500000
2022-02-01,Pedro,2000,12,2022-02-01,166.666667


In [10]:
deuda_mensual = creditos.groupby([pd.Grouper(freq="M"), "Cliente"])["Cuota Mensual"].sum()
deuda_mensual

fecha_mes   Cliente
2022-01-31  Ana        333.333333
            Juan        83.333333
            Pedro      187.500000
2022-02-28  Juan       208.333333
            Pedro      166.666667
2022-03-31  Ana        250.000000
Name: Cuota Mensual, dtype: float64

In [11]:
deuda_mensual = deuda_mensual.reset_index()
deuda_mensual

,fecha_mes,Cliente,Cuota Mensual
0,2022-01-31,Ana,333.333333
1,2022-01-31,Juan,83.333333
2,2022-01-31,Pedro,187.500000
3,2022-02-28,Juan,208.333333
4,2022-02-28,Pedro,166.666667
5,2022-03-31,Ana,250.000000


## 3. 